In [1]:
# Cell 1: Setup and Imports - Discover ALL Codes
import sys
import time
import numpy as np
import warnings
from IPython.display import clear_output

clear_output(wait=True)
warnings.filterwarnings('ignore')

# Force reload of modules
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

import stim
from qectostim.codes import discover_all_codes
from qectostim.codes.abstract_code import StabilizerCode
from qectostim.codes.abstract_css import CSSCode
from qectostim.experiments.memory import CSSMemoryExperiment, StabilizerMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

# Discover ALL codes including subsystem, floquet, and QLDPC
discovered = discover_all_codes(
    max_qubits=50, 
    include_qldpc=True,
    include_subsystem=True,
    include_floquet=True
)

# Categorize codes by type - use dict to prevent duplicates
css_codes = {}
non_css_codes = {}
subsystem_codes = {}
floquet_codes = {}
qldpc_codes = {}

for name, code in discovered.items():
    if 'Floquet' in name or 'Honeycomb' in name or 'ISG' in name:
        floquet_codes[name] = code
    elif 'Subsystem' in name or 'Gauge' in name or 'Bacon' in name:
        subsystem_codes[name] = code
    elif any(x in name for x in ['HGP', 'BB', 'GB', 'Hypergraph', 'Bicycle', 'Lifted', 'Fiber']):
        qldpc_codes[name] = code
    elif hasattr(code, 'hx') and hasattr(code, 'hz'):
        css_codes[name] = code
    elif hasattr(code, 'stabilizer_matrix'):
        non_css_codes[name] = code

# Combine ALL codes with type markers
all_codes = {}
for name, code in css_codes.items():
    all_codes[name] = ('CSS', code)
for name, code in non_css_codes.items():
    all_codes[name] = ('Non-CSS', code)
for name, code in subsystem_codes.items():
    all_codes[name] = ('Subsystem', code)
for name, code in floquet_codes.items():
    all_codes[name] = ('Floquet', code)
for name, code in qldpc_codes.items():
    all_codes[name] = ('QLDPC', code)

# Build output as single string to prevent duplication
output = []
output.append("="*70)
output.append("DECODER SMOKE TEST - ALL CODE TYPES (NO SKIPPING)")
output.append("="*70)
output.append(f"\nTotal discovered: {len(discovered)} codes")
output.append(f"  CSS:       {len(css_codes)}")
output.append(f"  Non-CSS:   {len(non_css_codes)}")
output.append(f"  Subsystem: {len(subsystem_codes)}")
output.append(f"  Floquet:   {len(floquet_codes)}")
output.append(f"  QLDPC:     {len(qldpc_codes)}")

for title, codes_dict in [("CSS Codes", css_codes), ("Non-CSS Codes", non_css_codes),
                           ("Subsystem Codes", subsystem_codes), ("Floquet Codes", floquet_codes),
                           ("QLDPC Codes", qldpc_codes)]:
    output.append(f"\n{title}:")
    output.append(f"{'Code Name':<40} {'n':>4} {'k':>3} {'d':>3}")
    output.append("-"*55)
    for name, code in codes_dict.items():
        d = code.metadata.get('distance', '?')
        output.append(f"  {name:<38} {code.n:>4} {code.k:>3} {d:>3}")
    output.append(f"\n{len(codes_dict)} codes")

print("\n".join(output))

DECODER SMOKE TEST - ALL CODE TYPES (NO SKIPPING)

Total discovered: 32 codes
  CSS:       18
  Non-CSS:   5
  Subsystem: 3
  Floquet:   1
  QLDPC:     5

CSS Codes:
Code Name                                   n   k   d
-------------------------------------------------------
  FourQubit422_[[4,2,?]]                    4   2   2
  C6                                        6   2   2
  Steane_713                                7   1   3
  Shor_91                                   9   1   3
  ReedMuller_15_1_3                        15   1   3
  ToricCode_3x3                            18   2   3
  Hamming_CSS_7                             7   1   3
  Code_832                                  8   3   2
  RotatedSurface_[[9,1,3]]                  9   1   3
  RotatedSurface_[[25,1,5]]                25   1   5
  TriangularColour_d3                       7   1   3
  HexagonalColour_d3                       17   7   3
  XZZX_Surface_3                            9   1   3
  XZZX_Surface_5      

In [2]:
# Cell 2: Load Available Decoders

decoder_classes = {}

# PyMatching
try:
    from qectostim.decoders.pymatching_decoder import PyMatchingDecoder
    decoder_classes['PyMatching'] = PyMatchingDecoder
except: pass

# Fusion Blossom
try:
    from qectostim.decoders.fusion_blossom_decoder import FusionBlossomDecoder
    decoder_classes['FusionBlossom'] = FusionBlossomDecoder
except: pass

# Belief Matching
try:
    from qectostim.decoders.belief_matching import BeliefMatchingDecoder
    decoder_classes['BeliefMatching'] = BeliefMatchingDecoder
except: pass

# BP-OSD
try:
    from qectostim.decoders.bp_osd import BPOSDDecoder
    decoder_classes['BPOSD'] = BPOSDDecoder
except: pass

# Tesseract
try:
    from qectostim.decoders.tesseract_decoder import TesseractDecoder
    decoder_classes['Tesseract'] = TesseractDecoder
except: pass

# Union Find (fallback to PyMatching)
try:
    from qectostim.decoders.union_find_decoder import UnionFindDecoder
    decoder_classes['UnionFind'] = UnionFindDecoder
except: pass

print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

Loaded 6 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind']


In [3]:
# Cell 3: Test Helper Function - NO SKIPPING

def test_decoder_on_code(code, code_type, decoder_name, decoder_class, p=0.01, shots=1000, rounds=3):
    """Test a decoder on ANY code type and return results dict.
    
    This function tests ALL codes - no skipping for any code type.
    If a code fails, it's marked as FAIL (not SKIP).
    """
    result = {
        'status': 'UNKNOWN',
        'ler': None,
        'ler_no_decode': None,
        'time_ms': None,
        'warnings': [],
        'error': None
    }
    
    try:
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        
        # Use appropriate experiment class based on code type
        if code_type == 'CSS':
            exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        else:
            # Non-CSS, Subsystem, Floquet, QLDPC all use StabilizerMemoryExperiment
            exp = StabilizerMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        
        circuit = noise.apply(exp.to_stim())
        
        # Try to build DEM - with fallback for decomposition failures
        try:
            dem = circuit.detector_error_model(decompose_errors=True)
        except Exception as e1:
            try:
                dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
            except Exception as e2:
                # If DEM fails, this is a real failure - NOT a skip
                result['status'] = 'FAIL'
                result['error'] = f"DEM failed: {str(e2)[:40]}"
                return result
        
        # Sample
        sampler = dem.compile_sampler()
        raw = sampler.sample(shots, bit_packed=False)
        
        if isinstance(raw, tuple):
            det_samples = np.asarray(raw[0], dtype=np.uint8)
            obs_samples = np.asarray(raw[1], dtype=np.uint8)
        else:
            arr = np.asarray(raw, dtype=np.uint8)
            det_samples = arr[:, :dem.num_detectors]
            obs_samples = arr[:, dem.num_detectors:]
        
        if obs_samples.shape[1] > 0:
            result['ler_no_decode'] = float(obs_samples[:, 0].mean())
        
        # Create decoder
        if decoder_name in ['PyMatching', 'FusionBlossom']:
            decoder = decoder_class(dem)
        else:
            decoder = decoder_class(dem=dem)
        
        # Decode
        start = time.time()
        corrections = decoder.decode_batch(det_samples)
        result['time_ms'] = (time.time() - start) * 1000
        
        corrections = np.asarray(corrections, dtype=np.uint8)
        if corrections.ndim == 1:
            corrections = corrections.reshape(-1, max(1, dem.num_observables))
        
        if obs_samples.shape[1] > 0:
            logical_errors = (corrections[:, 0] ^ obs_samples[:, 0]).astype(np.uint8)
            result['ler'] = float(logical_errors.mean())
        
        result['status'] = 'OK'
        
        # Warnings
        ler, ler_nd = result['ler'], result['ler_no_decode']
        if ler is not None:
            if ler < 1e-6 and p > 0.001:
                result['warnings'].append('LER≈0')
            if ler_nd is not None and ler >= ler_nd and code.metadata.get('distance', 0) >= 3:
                result['warnings'].append('No improvement')
        
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = str(e)[:50]
    
    return result

print("✓ Test helper defined (tests ALL code types - NO SKIPPING)")

✓ Test helper defined (tests ALL code types - NO SKIPPING)


In [4]:
# Cell 4: DECODER × CODE COMPATIBILITY MATRIX - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

p = 0.01
shots = 1000

# Store results - reinitialize to be safe
full_results = {}

# Build output lines to print at end (prevents output duplication)
output_lines = []
output_lines.append("="*120)
output_lines.append("DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES (NO SKIPPING)")
output_lines.append("="*120)
output_lines.append(f"\nTesting {len(all_codes)} decoder/code combinations at p={p}, {shots} shots...")
output_lines.append("Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:^12}"
output_lines.append(header)
output_lines.append("-" * len(header))

# Process each code ONCE
for code_name, (code_type, code) in all_codes.items():
    row = f"{code_name:<35} {code_type:<10}"
    full_results[code_name] = {'type': code_type}
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=code,
            code_type=code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=shots
        )
        full_results[code_name][dec_name] = result
        
        if result['status'] == 'OK':
            ler = result['ler']
            if result['warnings']:
                cell = f"⚠️{ler:.4f}"
            else:
                cell = f"✓ {ler:.4f}"
        else:
            cell = f"✗ FAIL"
        
        row += f" | {cell:^12}"
    
    output_lines.append(row)

output_lines.append("-" * len(header))
output_lines.append(f"\nLegend: ✓=pass, ⚠️=warning, ✗=fail")
output_lines.append(f"Total codes tested: {len(full_results)}")

# Single print statement
print("\n".join(output_lines))

DECODER × CODE COMPATIBILITY MATRIX - ALL CODE TYPES (NO SKIPPING)

Testing 32 decoder/code combinations at p=0.01, 1000 shots...
Code types: CSS, Non-CSS, Subsystem, Floquet, QLDPC

Code                                Type       |  PyMatching  |  FusionBlos  |  BeliefMatc  |    BPOSD     |  Tesseract   |  UnionFind  
----------------------------------------------------------------------------------------------------------------------------------------
FourQubit422_[[4,2,?]]              CSS        |   ✓ 0.0580   |   ✓ 0.0580   |   ✓ 0.0580   |   ✓ 0.0530   |   ✓ 0.0560   |   ✓ 0.0440  
C6                                  CSS        |   ✓ 0.0730   |   ✓ 0.0920   |   ✓ 0.0930   |   ✓ 0.0920   |   ✓ 0.0760   |   ✓ 0.0770  
Steane_713                          CSS        |   ✓ 0.0840   |   ✓ 0.0920   |   ✓ 0.0710   |   ✓ 0.0560   |   ✓ 0.0190   |   ✓ 0.0770  
Shor_91                             CSS        |   ✓ 0.0060   |   ✓ 0.0020   |   ✓ 0.0020   |   ✓ 0.0030   |   ✓ 0.0050   |   ✓ 0.00

In [5]:
# Cell 5: LER COMPARISON TABLE - ALL CODES
from IPython.display import clear_output
clear_output(wait=True)

print("="*130)
print("LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES")
print("="*130)
print(f"\nTotal codes in all_codes: {len(all_codes)}")
print(f"Total codes in full_results: {len(full_results)}")
print("\nLower is better. Best decoder for each code highlighted.\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<35} | {'Type':<10} | {'d':>2} | {'No-decode':>10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
header += " | Best"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    
    # Get code object for distance
    code = all_codes.get(code_name, (None, None))[1]
    d = code.metadata.get('distance', '?') if code else '?'
    
    # Get no-decode LER
    ler_no_decode = None
    for key, res in code_results.items():
        if isinstance(res, dict) and res.get('ler_no_decode') is not None:
            ler_no_decode = res['ler_no_decode']
            break
    
    nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else 'N/A'
    row = f"{code_name:<35} | {code_type:<10} | {str(d):>2} | {nd_str:>10}"
    
    # Find best decoder
    best_ler = float('inf')
    best_decoder = None
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        ler = res.get('ler') if isinstance(res, dict) else None
        if ler is not None:
            ler_str = f"{ler:.4f}"
            if ler < best_ler:
                best_ler = ler
                best_decoder = dec_name
        else:
            ler_str = 'FAIL'
        row += f" | {ler_str:>10}"
    
    best_str = best_decoder[:10] if best_decoder else 'N/A'
    row += f" | {best_str}"
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

LER COMPARISON TABLE (p=0.01) - ALL CODE TYPES

Total codes in all_codes: 32
Total codes in full_results: 32

Lower is better. Best decoder for each code highlighted.

Code                                | Type       |  d |  No-decode | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind | Best
-------------------------------------------------------------------------------------------------------------------------------------------------------
BB_tiny_18                          | QLDPC      |  ? |        N/A |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL | N/A
BareAncilla_713                     | Non-CSS    |  3 |        N/A |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL | N/A
C6                                  | CSS        |  2 |     0.0730 |     0.0730 |     0.0920 |     0.0930 |     0.0920 |     0.0760 |     0.0770 | PyMatching
Code_832                            | CSS        |  2 |     0.0760 |

In [6]:
# Cell 6: DECODER SPEED COMPARISON
from IPython.display import clear_output
clear_output(wait=True)

print("="*110)
print("DECODER SPEED COMPARISON (time in ms for 1000 shots)")
print("="*110)
print(f"\nTotal codes in full_results: {len(full_results)}\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<30} {'Type':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
print(header)
print("-" * len(header))

# Use full_results directly since it has all the data from Cell 4
for code_name in sorted(full_results.keys()):
    code_results = full_results[code_name]
    code_type = code_results.get('type', 'Unknown')
    row = f"{code_name:<30} {code_type:<10}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        time_ms = res.get('time_ms') if isinstance(res, dict) else None
        time_str = f"{time_ms:.1f}" if time_ms else 'FAIL'
        row += f" | {time_str:>10}"
    
    print(row)

print("-" * len(header))
print(f"\nTotal rows: {len(full_results)}")

DECODER SPEED COMPARISON (time in ms for 1000 shots)

Total codes in full_results: 32

Code                           Type       | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind
-----------------------------------------------------------------------------------------------------------------------
BB_tiny_18                     QLDPC      |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL
BareAncilla_713                Non-CSS    |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL
C6                             CSS        |        0.1 |        2.7 |        5.0 |        5.5 |       11.5 |        0.1
Code_832                       CSS        |        0.3 |        5.7 |       19.0 |       48.5 |       13.0 |        0.3
FiberBundle_f3                 QLDPC      |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL |       FAIL
FourQubit422_[[4,2,?]]         CSS        |        0.1 |        2.1 |    

In [7]:
# Cell 7: NOISE LEVEL SCALING TEST

print("="*110)
print("NOISE LEVEL SCALING TEST")
print("="*110)

# Pick a representative CSS code (RotatedSurface d=3) 
test_code_name = None
test_code_type = None
for name, (ctype, code) in all_codes.items():
    if 'RotatedSurface' in name and code.metadata.get('distance') == 3:
        test_code_name = name
        test_code_type = ctype
        break

if test_code_name is None:
    test_code_name = list(all_codes.keys())[0]
    test_code_type = all_codes[test_code_name][0]

test_code = all_codes[test_code_name][1]
print(f"\nTesting {test_code_name} ({test_code_type}) across noise levels with all decoders...\n")

noise_levels = [0.001, 0.005, 0.01, 0.02]

# Header
dec_names = list(decoder_classes.keys())
header = f"{'p':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:12]:>12}"
header += " | No-decode"
print(header)
print("-" * len(header))

for p in noise_levels:
    row = f"{p:<10.4f}"
    ler_no_decode = None
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=test_code,
            code_type=test_code_type,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=2000,
            rounds=3
        )
        
        if ler_no_decode is None and result.get('ler_no_decode'):
            ler_no_decode = result['ler_no_decode']
        
        ler = result.get('ler')
        ler_str = f"{ler:.6f}" if ler else 'FAIL'
        row += f" | {ler_str:>12}"
    
    nd_str = f"{ler_no_decode:.6f}" if ler_no_decode else 'N/A'
    row += f" | {nd_str}"
    print(row)

print("-" * len(header))

NOISE LEVEL SCALING TEST

Testing RotatedSurface_[[9,1,3]] (CSS) across noise levels with all decoders...

p          |   PyMatching | FusionBlosso | BeliefMatchi |        BPOSD |    Tesseract |    UnionFind | No-decode
----------------------------------------------------------------------------------------------------------------
0.0010     |         FAIL |         FAIL |         FAIL |     0.000500 |         FAIL |         FAIL | 0.008500
0.0010     |         FAIL |         FAIL |         FAIL |     0.000500 |         FAIL |         FAIL | 0.008500
0.0050     |     0.006000 |     0.005000 |     0.005000 |     0.007000 |     0.004000 |     0.004500 | 0.055000
0.0050     |     0.006000 |     0.005000 |     0.005000 |     0.007000 |     0.004000 |     0.004500 | 0.055000
0.0100     |     0.018500 |     0.016500 |     0.021000 |     0.017000 |     0.016500 |     0.020500 | 0.117000
0.0100     |     0.018500 |     0.016500 |     0.021000 |     0.017000 |     0.016500 |     0.020500 | 0.11

In [8]:
# Cell 8: FINAL SUMMARY - ALL CODE TYPES

print("="*100)
print("FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES")
print("="*100)

# Count results by status
passed = 0
with_warnings = 0
failed = 0
total_tests = 0

# Track by code type
results_by_type = {'CSS': {'pass': 0, 'warn': 0, 'fail': 0},
                   'Non-CSS': {'pass': 0, 'warn': 0, 'fail': 0},
                   'Subsystem': {'pass': 0, 'warn': 0, 'fail': 0},
                   'Floquet': {'pass': 0, 'warn': 0, 'fail': 0},
                   'QLDPC': {'pass': 0, 'warn': 0, 'fail': 0}}

failed_tests = []

for code_name, code_results in full_results.items():
    code_type = code_results.get('type', 'Unknown')
    for dec_name in decoder_classes.keys():
        res = code_results.get(dec_name, {})
        if isinstance(res, dict):
            total_tests += 1
            if res.get('status') == 'OK':
                if res.get('warnings'):
                    with_warnings += 1
                    results_by_type[code_type]['warn'] += 1
                else:
                    passed += 1
                    results_by_type[code_type]['pass'] += 1
            else:
                failed += 1
                results_by_type[code_type]['fail'] += 1
                failed_tests.append((code_name, dec_name, res.get('error', 'unknown')))

print(f"\n📊 OVERALL RESULTS:")
print(f"   Total decoder × code tests: {total_tests}")
print(f"   ✓ Passed:   {passed} ({100*passed/total_tests:.1f}%)")
print(f"   ⚠ Warnings: {with_warnings} ({100*with_warnings/total_tests:.1f}%)")
print(f"   ✗ Failed:   {failed} ({100*failed/total_tests:.1f}%)")

print(f"\n📊 RESULTS BY CODE TYPE:")
for ctype, counts in results_by_type.items():
    total = counts['pass'] + counts['warn'] + counts['fail']
    if total > 0:
        print(f"   {ctype:>10}: {counts['pass']} pass, {counts['warn']} warn, {counts['fail']} fail (total {total})")

print(f"\n🔧 DECODERS TESTED: {len(decoder_classes)}")
for dec_name in decoder_classes.keys():
    print(f"   - {dec_name}")

print(f"\n📦 CODES TESTED BY TYPE:")
for code_type, codes_dict in [('CSS', css_codes), ('Non-CSS', non_css_codes), 
                               ('Subsystem', subsystem_codes), ('Floquet', floquet_codes),
                               ('QLDPC', qldpc_codes)]:
    if codes_dict:
        print(f"\n   {code_type} ({len(codes_dict)} codes):")
        for name, code in codes_dict.items():
            d = code.metadata.get('distance', '?')
            print(f"     - {name}: [[{code.n},{code.k},{d}]]")

print("\n" + "="*100)
if failed == 0:
    print("✓ ALL TESTS PASSED (no failures)")
else:
    print(f"⚠ {failed} TESTS FAILED:")
    for code_name, dec_name, error in failed_tests[:20]:  # Show first 20
        print(f"   - {code_name} × {dec_name}: {error}")
    if len(failed_tests) > 20:
        print(f"   ... and {len(failed_tests) - 20} more")
print("="*100)

FINAL DECODER SMOKE TEST SUMMARY - ALL CODE TYPES

📊 OVERALL RESULTS:
   Total decoder × code tests: 192
   ✓ Passed:   81 (42.2%)
   ⚠ Warnings: 27 (14.1%)
   ✗ Failed:   84 (43.8%)

📊 RESULTS BY CODE TYPE:
          CSS: 81 pass, 27 warn, 0 fail (total 108)
      Non-CSS: 0 pass, 0 warn, 30 fail (total 30)
    Subsystem: 0 pass, 0 warn, 18 fail (total 18)
      Floquet: 0 pass, 0 warn, 6 fail (total 6)
        QLDPC: 0 pass, 0 warn, 30 fail (total 30)

🔧 DECODERS TESTED: 6
   - PyMatching
   - FusionBlossom
   - BeliefMatching
   - BPOSD
   - Tesseract
   - UnionFind

📦 CODES TESTED BY TYPE:

   CSS (18 codes):
     - FourQubit422_[[4,2,?]]: [[4,2,2]]
     - C6: [[6,2,2]]
     - Steane_713: [[7,1,3]]
     - Shor_91: [[9,1,3]]
     - ReedMuller_15_1_3: [[15,1,3]]
     - ToricCode_3x3: [[18,2,3]]
     - Hamming_CSS_7: [[7,1,3]]
     - Code_832: [[8,3,2]]
     - RotatedSurface_[[9,1,3]]: [[9,1,3]]
     - RotatedSurface_[[25,1,5]]: [[25,1,5]]
     - TriangularColour_d3: [[7,1,3]]
     - 